# Recolección de datos

In [2]:
import requests
import pandas as pd

## Clasificación de países por nivel de ingreso

In [3]:
#Aqui vemos la lista de niveles de Ingreso y sus respectivos códigos
url = 'http://api.worldbank.org/v2/incomelevel' 
args = {"format":"json", "prefix":"Getdata"}
dic_Income_Level = requests.get(url, params=args)
datos = dic_Income_Level.json()[1]
data = pd.json_normalize(datos)
data[['value', 'id']]

,value,id
0,High income,HIC
1,Not classified,INX
2,Low income,LIC
3,Lower middle income,LMC
4,Low & middle income,LMY
5,Middle income,MIC
6,Upper middle income,UMC


In [4]:
incomes = ['HIC', 'UMC', 'LMC', 'LIC','INX']
nivel_ingreso = pd.DataFrame()
for income in incomes:
    url = 'http://api.worldbank.org/v2/country' #Aqui vemos la lista de niveles de Ingreso y sus respectivos códigos
    args = {"format":"json", "prefix":"Getdata", "incomelevel":income}
    dic_Income_Level = requests.get(url, params=args)
    datos = dic_Income_Level.json()[1]
    data = pd.json_normalize(datos)
    #data = data[['name', 'id', 'region.value', 'incomeLevel.value']]
    nivel_ingreso = pd.concat([nivel_ingreso, data], ignore_index=True)


In [89]:
nivel_ingreso

,name,id,region.value,incomeLevel.value
0,Aruba,ABW,Latin America & Caribbean,High income
1,Andorra,AND,Europe & Central Asia,High income
2,United Arab Emirates,ARE,Middle East & North Africa,High income
3,Antigua and Barbuda,ATG,Latin America & Caribbean,High income
4,Australia,AUS,East Asia & Pacific,High income
...,...,...,...,...
174,Togo,TGO,Sub-Saharan Africa,Low income
175,Uganda,UGA,Sub-Saharan Africa,Low income
176,"Yemen, Rep.",YEM,Middle East & North Africa,Low income
177,Zambia,ZMB,Sub-Saharan Africa,Low income


## Obtención de distintos indicadores

In [3]:
def consultar(pais: str, indicador:str, pagina:int = 1):

    '''Función que contacta a la API y devuelve la respuesta
    que brinda la misma en formato json'''

    # Página de la api y path al recurso solicitado
    api_url = 'http://api.worldbank.org/v2/es'
    path = f'/country/{pais}/indicator/{indicador}'
    url = api_url + path

    # Creamos el diccionario con los parametros 
    # para el método get
    args= {
        "date":'1990:2020',
        'page':pagina,
        "per_page":1000,
        "format":"json",
        "prefix":"Getdata",
    }
   
    return requests.get(url,params=args).json()

In [4]:
def consultar_por_pais():
    '''Función que retorna un código de país,
    para, en caso que se quiera o se solicite,
    consultar en tiempo real la API'''
    paises=pd.read_parquet('../datasets/paises.parquet')

    seleccion = input("Ingrese un nombre de país: ")
    try:
        pais = paises.loc[paises.name.str.contains(seleccion, case=False),'id'].values[0]
    except:
        print("No se encontró el país, intente de nuevo")

In [5]:
# Estos indicadores fueron seleccionados 
# de forma manual al final del documento
indicadores = pd.read_csv('../datasets/indicadores_seleccionados.csv')

tuplas = [(a,b) for a,b in zip(indicadores.id.to_list(), 
          indicadores.name.to_list())]

In [12]:
def carga_incremental(pais, indicador):
    '''Función que a partir de un país y un indicador 
    llama a consultar y establece qué tipo de contenido tiene
    según eso devuelve o no un dataframe con todos los datos'''

    consulta = consultar(pais, indicador)
    try:
        # La primera parte de la respuesta nos indica en 
        # cuantas páginas de encuentra la información
        paginas = consulta[0]["pages"]

        # La segunda parte nos retorna una lista de 
        # diccionarios con la información que queríamos
        datos=consulta[1]

    except:
        print('No hay datos para:', indicador, pais)
        pass
    else:
        if paginas >= 1:
            # Agregamos los valores de las otras páginas a
            # nuestra lista de diccionarios
            for pagina in range(2,paginas+1):
                datos.extend(consultar(pais, indicador, pagina)[1])

            # Creo el DataFrame con todos los datos
            data = pd.json_normalize(datos)
            return data
        return pd.DataFrame(['error'],columns=['no_data'])


In [ ]:
for indicador, _ in tuplas:
    print(indicador)
    datos = carga_incremental(pais='all', indicador=indicador)

    # Guardo el dataframe resultante
    datos.to_parquet(f'../datasets/df_TWB_{indicador}.parquet')
    print('Guardado')


# ETL

## Búsqueda de indicadores

La lista de indicadores bajo estudio se selecciona de forma manual a partir de los datos que indican distintas tesis de estudio. De todas formas los indicadores pasarán por un tamiz de análisis exploratorio

In [19]:
indicadores_todos = pd.read_csv('../datasets/indicadores.csv')

# Se completó los valores faltantes con 'na'
# para poder buscar sobre el dataframe
indicadores_todos.fillna('na',inplace=True)

In [37]:
interes = input("Indicador de interés: ")
indicadores_todos.loc[
     indicadores_todos.name.str.contains(interes,case=False),
     ['id','name','sourceNote']
     ]#.to_csv('temp.csv')

,id,name,sourceNote
17810,SP.POP.TOTL.FE.IN,"Población, mujeres","Población, mujeres"
17811,SP.POP.TOTL.FE.ZS,"Población, mujeres (% del total)",La población femenina es el porcentaje de la p...


In [26]:
interes = input("Indicador de interés: ")
seleccion = indicadores_todos.loc[indicadores_todos.id == interes,
     ['id','name','sourceNote']]
seleccion

,id,name,sourceNote
17815,SP.POP.TOTL.MA.ZS,"Población, hombres (% del total)",na


In [6]:
indicador_selec = pd.DataFrame()

In [108]:
indicador_selec = pd.concat([indicador_selec, seleccion], 
                            ignore_index=True)

## Exploración de los Datasets obtenidos

In [7]:
codigos = [i for i, j in tuplas]
nombres = [j for i, j in tuplas]

columnas=['esperanza_vida_total', 'esperanza_vida_mujeres','esperanza_vida_varones','índice_gini','gasto_púb_educacion_pje','duración_educ_obligatoria','pib_pc_usd_actuales','pib_ppa_prec_inter','capacidad_estadística','población_mujeres_pje','población_hombres_pje','pib_pc_prec_inter']
print(len(columnas))
print(len(nombres))

12
12


In [8]:
import os
directorio = '../datasets/'

with os.scandir(directorio) as ficheros:

    trabajo=pd.read_parquet('../datasets/df_TWB_SP.DYN.LE00.IN.parquet')[['date','countryiso3code']]

    for fichero in ficheros:
        if fichero.name.startswith('df_TWB'):

            # obtengo el cógigo de indicador que se encuentra en el nombre del fichero
            codigo_fichero = fichero.name[7:-8]
            # busco el código en mi lista de códigos 
            # y procedo a renombrar la columna de interés
            if codigo_fichero in codigos:
                df=pd.read_parquet(directorio+fichero.name)

                # obtengo el índice en mi lista de códigos que 
                # se corresponde con el índice en mi lista de nombres
                indice_codigo = codigos.index(codigo_fichero)
                #df.rename(columns={'value':columnas[indice_codigo]},inplace=True)
                trabajo[columnas[indice_codigo]]=df.value
                
                

In [22]:
trabajo = trabajo.merge(
                        nivel_ingreso[['id','incomeLevel.value']], 
                        left_on='countryiso3code',
                        right_on='id',
                        )

In [83]:
trabajo

,date,countryiso3code,pib_ppa_prec_inter,población_mujeres_pje,esperanza_vida_total,población_hombres_pje,esperanza_vida_mujeres,esperanza_vida_varones,pib_pc_prec_inter,pib_pc_usd_actuales,id,incomeLevel.value
0,2020,AFG,8.091174e+10,48.684520,65.173,51.315480,66.744,63.708,2078.479082,516.747871,AFG,Low income
1,2019,AFG,8.187310e+10,48.662395,64.833,51.337605,66.388,63.382,2152.190243,494.179350,AFG,Low income
2,2018,AFG,7.740652e+10,48.635847,64.486,51.364153,66.026,63.047,2082.392197,485.668419,AFG,Low income
3,2017,AFG,7.471192e+10,48.611616,64.130,51.388384,65.656,62.701,2058.400221,516.679862,AFG,Low income
4,2016,AFG,7.009796e+10,48.599668,63.763,51.400332,65.275,62.343,1981.118069,512.012778,AFG,Low income
...,...,...,...,...,...,...,...,...,...,...,...,...
5544,1994,ZMB,1.255557e+10,50.301818,44.983,49.698182,46.808,43.264,1415.550409,412.260752,ZMB,Low income
5545,1993,ZMB,1.345348e+10,50.264033,45.919,49.735967,47.598,44.321,1554.151167,378.125559,ZMB,Low income
5546,1992,ZMB,1.230553e+10,50.223669,46.987,49.776331,48.521,45.513,1456.044324,376.498819,ZMB,Low income
5547,1991,ZMB,1.224327e+10,50.186754,48.125,49.813246,49.523,46.768,1484.633685,409.727275,ZMB,Low income


In [33]:
trabajo.isna().sum()

date                            0
countryiso3code                 0
pib_ppa_prec_inter            665
población_mujeres_pje         443
esperanza_vida_total          305
población_hombres_pje         443
duración_educ_obligatoria    1995
índice_gini                  4076
esperanza_vida_mujeres        305
esperanza_vida_varones        305
gasto_púb_educacion_pje      2646
capacidad_estadística        3360
pib_pc_prec_inter             668
pib_pc_usd_actuales           369
id                              0
incomeLevel.value               0
dtype: int64

Al ser nuestro objetivo la esperanza de vida, no sería recomendable inventar esos datos con un promedio, ni nada parecido. por lo que vamos a limpiar todas las filas que tengan valores nulos en la esperanza de vida al nacer

In [46]:
trabajo.dropna(subset=['esperanza_vida_total'], inplace=True)

Verificamos nuevamente los datos

In [60]:
trabajo.isna().sum()

date                            0
countryiso3code                 0
pib_ppa_prec_inter            501
población_mujeres_pje         166
esperanza_vida_total            0
población_hombres_pje         166
duración_educ_obligatoria    1858
índice_gini                  3772
esperanza_vida_mujeres          0
esperanza_vida_varones          0
gasto_púb_educacion_pje      2414
capacidad_estadística        3118
pib_pc_prec_inter             504
pib_pc_usd_actuales           276
id                              0
incomeLevel.value               0
dtype: int64

Vamos a eliminar todas aquellas columnas que tengan más del 30% de datos núlos. Dado que de trabajar sobre ellas con alguna estratégia de imputación no sería adecuado.

In [78]:
(trabajo.isna().sum()/trabajo.shape[0])*100

date                          0.000000
countryiso3code               0.000000
pib_ppa_prec_inter            9.553776
población_mujeres_pje         3.165523
esperanza_vida_total          0.000000
población_hombres_pje         3.165523
duración_educ_obligatoria    35.430969
índice_gini                  71.929825
esperanza_vida_mujeres        0.000000
esperanza_vida_varones        0.000000
gasto_púb_educacion_pje      46.033562
capacidad_estadística        59.458429
pib_pc_prec_inter             9.610984
pib_pc_usd_actuales           5.263158
id                            0.000000
incomeLevel.value             0.000000
dtype: float64

In [79]:
trabajo.drop(columns=[  'índice_gini',
                        'capacidad_estadística', 
                        'gasto_púb_educacion_pje', 
                        'duración_educ_obligatoria'],
            inplace=True
            )

In [82]:
trabajo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5244 entries, 0 to 5548
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   date                    5244 non-null   object 
 1   countryiso3code         5244 non-null   object 
 2   pib_ppa_prec_inter      4743 non-null   float64
 3   población_mujeres_pje   5078 non-null   float64
 4   esperanza_vida_total    5244 non-null   float64
 5   población_hombres_pje   5078 non-null   float64
 6   esperanza_vida_mujeres  5244 non-null   float64
 7   esperanza_vida_varones  5244 non-null   float64
 8   pib_pc_prec_inter       4740 non-null   float64
 9   pib_pc_usd_actuales     4968 non-null   float64
 10  id                      5244 non-null   object 
 11  incomeLevel.value       5244 non-null   object 
dtypes: float64(8), object(4)
memory usage: 532.6+ KB


In [88]:
corr_matrix = trabajo.corr()
corr_matrix["esperanza_vida_total"].sort_values(ascending=False)

esperanza_vida_total      1.000000
esperanza_vida_mujeres    0.994401
esperanza_vida_varones    0.993782
pib_pc_prec_inter         0.591415
pib_pc_usd_actuales       0.540825
pib_ppa_prec_inter        0.211011
población_hombres_pje     0.032236
población_mujeres_pje    -0.032236
Name: esperanza_vida_total, dtype: float64

Dividimos el dataset de trabajo en cuatro datasets para analizar según su nivel de ingresos

In [90]:
trabajo['incomeLevel.value'].value_counts()

Lower middle income    1550
Upper middle income    1427
High income            1368
Low income              868
Not classified           31
Name: incomeLevel.value, dtype: int64

In [91]:
ingreso_bajo = trabajo.loc[trabajo['incomeLevel.value'] == 'Low income']
ingreso_medio_bajo = trabajo.loc[trabajo['incomeLevel.value'] == 'Lower middle income']
ingreso_medio_alto = trabajo.loc[trabajo['incomeLevel.value'] == 'Upper middle income']
ingreso_alto = trabajo.loc[trabajo['incomeLevel.value'] == 'High income']


In [93]:
ingreso_alto.isnull().sum()

date                        0
countryiso3code             0
pib_ppa_prec_inter        224
población_mujeres_pje     148
esperanza_vida_total        0
población_hombres_pje     148
esperanza_vida_mujeres      0
esperanza_vida_varones      0
pib_pc_prec_inter         227
pib_pc_usd_actuales       105
id                          0
incomeLevel.value           0
dtype: int64